In [ ]:
import wandb
import random
from types import SimpleNamespace
wandb.finish()
wandb.login(key = 'b5d1fbca9d5170f54415e9c5a70ef09cee7a0aec')

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ma23m021 (ma23m021-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
from keras.datasets import fashion_mnist
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split

# fashion-MNIST dataset..................
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# normalizing input data...............
x_train = x_train.reshape(x_train.shape[0], -1) / 255.0
x_test = x_test.reshape(x_test.shape[0], -1) / 255.0

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# One-hot encoding function.............................
def onehot_encoding(y, num_classes=10):
    return np.eye(num_classes)[y]

# Activation functions................
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def relu_derivative(x):
    return (x > 0).astype(float)

# initializing weights and biases.......................................
def param_init(input_size, num_hidden_layers, num_neurons, output_size=10, init_method='xavier'):
    layers = [input_size] + [num_neurons] * num_hidden_layers + [output_size]
    weights = {}
    biases = {}

    # initialization choices.......................................
    for i in range(1, len(layers)):
        if init_method == 'xavier':
            weights[f'W{i}'] = np.random.randn(layers[i - 1], layers[i]) * np.sqrt(2 / layers[i - 1])
        elif init_method == 'random':
            weights[f'W{i}'] = np.random.randn(layers[i - 1], layers[i]) * 0.01
        else:
            raise ValueError("Invalid init_method! Use 'xavier' or 'random'.")

        biases[f'b{i}'] = np.zeros((1, layers[i]))

    return weights, biases


# Define activation functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

def relu(x):
    return np.maximum(0, x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# forward propagation........................................
def forwardd_propagation_(x, num_hidden_layers, num_neurons, weights=None, biases=None, output_size=10, activation_func='sigmoid'):
    input_size = x.shape[1]
    # initializing the parameters that is weights and bias.........................
    if weights is None or biases is None:
        weights, biases = param_init(input_size, num_hidden_layers, num_neurons, output_size)

    activations = {0: x}
    z_values = {}
# updation loop.......................................
    for i in range(1, len(weights) + 1):
        z_values[i] = np.dot(activations[i - 1], weights[f'W{i}']) + biases[f'b{i}']
        if i < len(weights):
            if activation_func == 'relu':
                activations[i] = relu(z_values[i])
            elif activation_func == 'tanh':
                activations[i] = tanh(z_values[i])
            else:
                activations[i] = sigmoid(z_values[i])
        else:
            activations[i] = softmax(z_values[i])

    return activations[len(weights)], activations, z_values, weights, biases


# Define activation function derivatives.........................
def relu_derivative(x):
    return np.where(x > 0, 1, 0)

def sigmoid_derivative(x):
    sig = 1 / (1 + np.exp(-x))
    return sig * (1 - sig)

def tanh_derivative(x):
    return 1 - np.tanh(x) ** 2

# Updated backward propagation....................................
def backwardd_propagation_(y, activations, z_values, weights, activation_function):
    grads = {}
    m = y.shape[0]

    def get_derivative(activation_func):
        if activation_func == 'relu':
            return relu_derivative
        elif activation_func == 'sigmoid':
            return sigmoid_derivative
        elif activation_func == 'tanh':
            return tanh_derivative

    hidden_derivative = get_derivative(activation_function)

    dz = activations[len(weights)] - y

    # Gradient calculation loop.................................
    for i in reversed(range(1, len(weights) + 1)):
        grads[f"dW{i}"] = np.dot(activations[i-1].T, dz) / m
        grads[f"db{i}"] = np.sum(dz, axis=0, keepdims=True) / m
        if i > 1:
            dz = np.dot(dz, weights[f'W{i}'].T) * hidden_derivative(z_values[i-1])

    return grads


# Update weights and bias using gradient descent..............................
def update_paramss_(weights, biases, grads, learningg_rate_):
    for i in range(1, len(weights) + 1):
        weights[f'W{i}'] -= learningg_rate_ * grads[f"dW{i}"]
        biases[f'b{i}'] -= learningg_rate_ * grads[f"db{i}"]
    return weights, biases

y_hat, _, _, _, _ = forwardd_propagation_(
    x=x_train,
    num_hidden_layers=3,
    num_neurons=128,
    activation_func='relu'
)


# printing probabilities and shape......................................
print("Probabilities:\n", y_hat[0])
print("Shape:", y_hat.shape)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Probabilities:
 [0.09345313 0.08493351 0.13893631 0.14191379 0.06422201 0.10989648
 0.13660777 0.04972838 0.07500006 0.10530856]
Shape: (48000, 10)


In [ ]:

# fashion-MNIST dataset...............................
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Normalize pixel values.........................
X_train = X_train.astype(np.float32) / 255.0
X_test = X_test.astype(np.float32) / 255.0

# flattening images.................................
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# One-hot encode labels.........................
y_train = onehot_encoding(y_train)
y_test = onehot_encoding(y_test)
'''
y_train = encoder.fit_transform(y_train.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1))'''

# Split data.................................
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)



# function for stochastic gradient optimizer .........................
def sgd(weights, biases, gradients, params):
    lr = params.get("learningg_rate_", 0.01)
    for i in range(1, len(weights) + 1):
        weights[f"W{i}"] -= lr * gradients[f"dW{i}"]
        biases[f"b{i}"] -= lr * gradients[f"db{i}"]
    return weights, biases



# function for Adam optimizer ..............................
def adam(weights, biases, gradients, params):
    lr, beta1, beta2, epsilon = params.get("learningg_rate_", 0.001), 0.9, 0.999, 1e-8

    # Initializing momentum and velocity dictionaries.............................
    m_w = params.setdefault("m_w", {f"W{i}": np.zeros_like(weights[f"W{i}"]) for i in range(1, len(weights) + 1)})
    v_w = params.setdefault("v_w", {f"W{i}": np.zeros_like(weights[f"W{i}"]) for i in range(1, len(weights) + 1)})
    m_b = params.setdefault("m_b", {f"b{i}": np.zeros_like(biases[f"b{i}"]) for i in range(1, len(biases) + 1)})
    v_b = params.setdefault("v_b", {f"b{i}": np.zeros_like(biases[f"b{i}"]) for i in range(1, len(biases) + 1)})
    t = params.setdefault("t", 0) + 1

    for i in range(1, len(weights) + 1):
        w_key, b_key = f"W{i}", f"b{i}"

        m_w[w_key] = beta1 * m_w[w_key] + (1 - beta1) * gradients[f'd{w_key}']
        v_w[w_key] = beta2 * v_w[w_key] + (1 - beta2) * (gradients[f'd{w_key}'] ** 2)
        m_b[b_key] = beta1 * m_b[b_key] + (1 - beta1) * gradients[f'd{b_key}']
        v_b[b_key] = beta2 * v_b[b_key] + (1 - beta2) * (gradients[f'd{b_key}'] ** 2)

        m_w_hat, v_w_hat = m_w[w_key] / (1 - beta1**t), v_w[w_key] / (1 - beta2**t)
        m_b_hat, v_b_hat = m_b[b_key] / (1 - beta1**t), v_b[b_key] / (1 - beta2**t)

        # Updateing  weights and biases...............................
        weights[w_key] -= lr * m_w_hat / (np.sqrt(v_w_hat) + epsilon)
        biases[b_key] -= lr * m_b_hat / (np.sqrt(v_b_hat) + epsilon)

    params["t"] = t  # Update timestep
    return weights, biases

#function for momentum optimizer ..............................
def momentum(weights, biases, gradients, params):
    lr, beta = params.get("learningg_rate_", 0.01), params.get("beta", 0.9)
    v_w = params.setdefault("v_w", {f"W{i}": np.zeros_like(weights[f"W{i}"]) for i in range(1, len(weights) + 1)})
    v_b = params.setdefault("v_b", {f"b{i}": np.zeros_like(biases[f"b{i}"]) for i in range(1, len(biases) + 1)})
    for i in range(1, len(weights) + 1):
        w_key_, b_key_ = f"W{i}", f"b{i}"
        v_w[w_key_] = beta * v_w[w_key_] + (1 - beta) * gradients[f"dW{i}"]
        v_b[b_key_] = beta * v_b[b_key_] + (1 - beta) * gradients[f"db{i}"]
        weights[w_key_] -= lr * v_w[w_key_]
        biases[b_key_] -= lr * v_b[b_key_]
    return weights, biases


# function for neaterov optimizer ..............................
def nesterov(weights, biases, gradients, params):
    lr, mu = params.get("learningg_rate_", 0.01), 0.9
    v_w = params.setdefault("v_w", {f"W{i}": np.zeros_like(weights[f"W{i}"]) for i in range(1, len(weights) + 1)})
    v_b = params.setdefault("v_b", {f"b{i}": np.zeros_like(biases[f"b{i}"]) for i in range(1, len(biases) + 1)})

    for i in range(1, len(weights) + 1):
        v_w[f"W{i}"] = mu * v_w[f"W{i}"] + lr * gradients[f"dW{i}"]
        v_b[f"b{i}"] = mu * v_b[f"b{i}"] + lr * gradients[f"db{i}"]
        weights[f"W{i}"] -= v_w[f"W{i}"]   # updating weights
        biases[f"b{i}"] -= v_b[f"b{i}"]
    return weights, biases

#function for rmsprop optimizer ..............................
def rmsprop(weights, biases, gradients, params):
    lr, beta, epsilon = params.get("learningg_rate_", 0.001), 0.9, 1e-8
    s_w = params.setdefault("s_w", {f"W{i}": np.zeros_like(weights[f"W{i}"]) for i in range(1, len(weights) + 1)})
    s_b = params.setdefault("s_b", {f"b{i}": np.zeros_like(biases[f"b{i}"]) for i in range(1, len(biases) + 1)})

    for i in range(1, len(weights) + 1):
        s_w[f"W{i}"] = beta * s_w[f"W{i}"] + (1 - beta) * gradients[f"dW{i}"]**2
        s_b[f"b{i}"] = beta * s_b[f"b{i}"] + (1 - beta) * gradients[f"db{i}"]**2
        weights[f"W{i}"] -= lr * gradients[f"dW{i}"] / (np.sqrt(s_w[f"W{i}"]) + epsilon)
        biases[f"b{i}"] -= lr * gradients[f"db{i}"] / (np.sqrt(s_b[f"b{i}"]) + epsilon)
    return weights, biases

#function for nadam optimizer ..............................
def nadam(weights, biases, gradients, params):
    lr, beta1, beta2, epsilon = params.get("learningg_rate_", 0.001), 0.9, 0.999, 1e-8

    # Initialize momentum and velocity dictionaries...............................
    m_w = params.setdefault("m_w", {f"W{i}": np.zeros_like(weights[f"W{i}"]) for i in range(1, len(weights) + 1)})
    v_w = params.setdefault("v_w", {f"W{i}": np.zeros_like(weights[f"W{i}"]) for i in range(1, len(weights) + 1)})
    m_b = params.setdefault("m_b", {f"b{i}": np.zeros_like(biases[f"b{i}"]) for i in range(1, len(biases) + 1)})
    v_b = params.setdefault("v_b", {f"b{i}": np.zeros_like(biases[f"b{i}"]) for i in range(1, len(biases) + 1)})
    t = params.setdefault("t", 0) + 1

    for i in range(1, len(weights) + 1):
        w_key, b_key = f"W{i}", f"b{i}"

        # Updating biased first moment estimate-momentum part............................
        m_w[w_key] = beta1 * m_w[w_key] + (1 - beta1) * gradients[f"dW{i}"]
        m_b[b_key] = beta1 * m_b[b_key] + (1 - beta1) * gradients[f"db{i}"]

        # Updating biased second moment estimate-RMSProp part...............................
        v_w[w_key] = beta2 * v_w[w_key] + (1 - beta2) * (gradients[f"dW{i}"] ** 2)
        v_b[b_key] = beta2 * v_b[b_key] + (1 - beta2) * (gradients[f"db{i}"] ** 2)

        # estimates of Bias-corrected first and second moments............................
        m_w_hat = m_w[w_key] / (1 - beta1**t)
        v_w_hat = v_w[w_key] / (1 - beta2**t)
        m_b_hat = m_b[b_key] / (1 - beta1**t)
        v_b_hat = v_b[b_key] / (1 - beta2**t)

        # Nesterov momentum:............................
        m_w_nesterov = beta1 * m_w_hat + (1 - beta1) * gradients[f"dW{i}"]
        m_b_nesterov = beta1 * m_b_hat + (1 - beta1) * gradients[f"db{i}"]

        # Updating weights and biases using Nadam...................
        weights[w_key] -= lr * m_w_nesterov / (np.sqrt(v_w_hat) + epsilon)
        biases[b_key] -= lr * m_b_nesterov / (np.sqrt(v_b_hat) + epsilon)

    params["t"] = t
    return weights, biases





In [ ]:



def training_model(X_train, Y_train, X_valid, Y_valid, num_hidden_layers, num_neurons, num_epochs=20, batch_size=32,
                   learningg_rate_=0.01, optimizer='adam', init_method='xavier', activation_func='relu',
                   loss_type='cross_entropy'):
    if Y_train.ndim == 1:
        Y_train = onehot_encoding(Y_train)
    if Y_valid.ndim == 1:
        Y_valid = onehot_encoding(Y_valid)

    # Initialize weights and biases
    input_size = X_train.shape[1]
    weights, biases = param_init(input_size, num_hidden_layers, num_neurons, init_method=init_method)

    # Optimizer parameters
    optimizer_params = {
        "learningg_rate_": learningg_rate_,
        "beta": 0.9,
        "beta1": 0.9,
        "beta2": 0.999,
        "eps": 1e-8
    }

    # Choose optimizer function
    optimizer_func = {
        'adam': adam, 'sgd': sgd, 'momentum': momentum,
        'nesterov': nesterov, 'rmsprop': rmsprop, 'nadam': nadam
    }.get(optimizer, adam)

    num_samples = X_train.shape[0]
    num_batches = num_samples // batch_size

    # Training loop
    for epoch in range(num_epochs):
        perm = np.random.permutation(num_samples)
        X_train, Y_train = X_train[perm], Y_train[perm]

        epoch_loss = 0
        for i in range(num_batches):
            batch_X = X_train[i * batch_size:(i + 1) * batch_size]
            batch_Y = Y_train[i * batch_size:(i + 1) * batch_size]

            # Forward pass
            probs, activations, z_values, weights, biases = forwardd_propagation_(
                batch_X, num_hidden_layers, num_neurons, weights, biases, activation_func
            )

            # Compute loss based on loss_type
            if loss_type == "cross_entropy":
                batch_loss = -np.mean(np.sum(batch_Y * np.log(probs + 1e-8), axis=1))
            elif loss_type == "mse":
                batch_loss = np.mean(np.sum((batch_Y - probs) ** 2, axis=1))
            else:
                raise ValueError(f"Unsupported loss type: {loss_type}")

            epoch_loss += batch_loss

            # Backward pass and update weights
            grads = backwardd_propagation_(batch_Y, activations, z_values, weights, activation_func)
            weights, biases = optimizer_func(weights, biases, grads, optimizer_params)

        # Validation phase
        valid_probs, _, _, _, _ = forwardd_propagation_(
            X_valid, num_hidden_layers, num_neurons, weights, biases, activation_func
        )

        # Validation loss and accuracy
        if loss_type == "cross_entropy":
            validation_loss_value = -np.mean(np.sum(Y_valid * np.log(valid_probs + 1e-8), axis=1))
        elif loss_type == "mse":
            validation_loss_value = np.mean(np.sum((Y_valid - valid_probs) ** 2, axis=1))

        val_accuracy_value = np.mean(np.argmax(valid_probs, axis=1) == np.argmax(Y_valid, axis=1))

        # Log to wandb
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": epoch_loss / num_batches,
            "val_loss": validation_loss_value,
            "val_accuracy": val_accuracy_value,
            "loss_type": loss_type
        })

        print(f"Epoch {epoch + 1}/{num_epochs}, {loss_type} - Train Loss: {epoch_loss / num_batches:.4f}, "
              f"Val Loss: {validation_loss_value:.4f}, Val Accuracy: {val_accuracy_value:.4f}")

    return weights, biases


In [ ]:
sweep_config = {
    "method": "bayes",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "parameters": {
        "epochs": {"values": [5, 10]},
        "num_hidden_layers": {"values": [3, 4, 5]},
        "hidden_layer_size": {"values": [32, 64, 128]},
        "learningg_rate_": {"values": [1e-3, 1e-4]},
        "optimizer": {"values": ["sgd", "momentum", "nesterov", "rmsprop", "adam", "nadam"]},
        "batch_size": {"values": [16, 32, 64]},
        "weight_initialization": {"values": ["random", "xavier"]},
        "activation_function": {"values": ["relu", "sigmoid", "tanh"]}
    }
}

# Cross-Entropy Sweep
sweep_config_ce = sweep_config.copy()
sweep_config_ce['parameters']['loss_type'] = {"value": "cross_entropy"}
print("Creating Cross-Entropy Sweep...")
sweep_id_ce = wandb.sweep(sweep=sweep_config_ce, project='MA23M021_A1_Q8_CROSS')
print("Cross-Entropy Sweep ID:", sweep_id_ce)

Creating Cross-Entropy Sweep...
Create sweep with ID: e8rqj5nt
Sweep URL: https://wandb.ai/ma23m021-iit-madras/MA23M021_A1_Q8_CROSS/sweeps/e8rqj5nt
Cross-Entropy Sweep ID: e8rqj5nt


In [ ]:
def main_():
    run = wandb.init(project='MA23M021_A1_Q8_CROSS', reinit=True)  # Specify the project name
    config = wandb.config

    # Train the model
    weights, biases = training_model(
        X_train=X_train,
        Y_train=y_train,
        X_valid=X_valid,
        Y_valid=y_valid,
        num_hidden_layers=config.num_hidden_layers,
        num_neurons=config.hidden_layer_size,
        num_epochs=config.epochs,
        batch_size=config.batch_size,
        learningg_rate_=config.learningg_rate_,
        optimizer=config.optimizer,
        init_method=config.weight_initialization,
        activation_func=config.activation_function,
        loss_type=config.loss_type
    )

    wandb.finish()  # Finish the run properly




In [ ]:
# Cross-Entropy Sweep
wandb.agent(sweep_id_ce, function=main_, count=50)



wandb: Agent Starting Run: yxa5kzia with config:
wandb: 	activation_function: sigmoid
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	hidden_layer_size: 64
wandb: 	learningg_rate_: 0.001
wandb: 	loss_type: cross_entropy
wandb: 	num_hidden_layers: 5
wandb: 	optimizer: sgd
wandb: 	weight_initialization: xavier


Epoch 1/10, cross_entropy - Train Loss: 2.3239, Val Loss: 2.3025, Val Accuracy: 0.0987
Epoch 2/10, cross_entropy - Train Loss: 2.3026, Val Loss: 2.3023, Val Accuracy: 0.1004
Epoch 3/10, cross_entropy - Train Loss: 2.3024, Val Loss: 2.3022, Val Accuracy: 0.1016
Epoch 4/10, cross_entropy - Train Loss: 2.3024, Val Loss: 2.3020, Val Accuracy: 0.1498
Epoch 5/10, cross_entropy - Train Loss: 2.3023, Val Loss: 2.3023, Val Accuracy: 0.1086
Epoch 6/10, cross_entropy - Train Loss: 2.3022, Val Loss: 2.3023, Val Accuracy: 0.0998
Epoch 7/10, cross_entropy - Train Loss: 2.3021, Val Loss: 2.3019, Val Accuracy: 0.0987
Epoch 8/10, cross_entropy - Train Loss: 2.3020, Val Loss: 2.3018, Val Accuracy: 0.2075
Epoch 9/10, cross_entropy - Train Loss: 2.3018, Val Loss: 2.3017, Val Accuracy: 0.1009
Epoch 10/10, cross_entropy - Train Loss: 2.3018, Val Loss: 2.3016, Val Accuracy: 0.1003


epoch,▁▂▃▃▄▅▆▆▇█
train_loss,█▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▄▂▁▁█▁▁
val_loss,█▇▅▄▇▆▃▃▂▁
epoch,10
loss_type,cross_entropy
train_loss,2.30177
val_accuracy,0.10025
val_loss,2.3016


wandb: Agent Starting Run: rk41ovil with config:
wandb: 	activation_function: sigmoid
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	hidden_layer_size: 128
wandb: 	learningg_rate_: 0.0001
wandb: 	loss_type: cross_entropy
wandb: 	num_hidden_layers: 4
wandb: 	optimizer: adam
wandb: 	weight_initialization: xavier


Epoch 1/10, cross_entropy - Train Loss: 1.4544, Val Loss: 0.9100, Val Accuracy: 0.6893
Epoch 2/10, cross_entropy - Train Loss: 0.7260, Val Loss: 0.6360, Val Accuracy: 0.7626
Epoch 3/10, cross_entropy - Train Loss: 0.5813, Val Loss: 0.5508, Val Accuracy: 0.7903
Epoch 4/10, cross_entropy - Train Loss: 0.5197, Val Loss: 0.5002, Val Accuracy: 0.8191
Epoch 5/10, cross_entropy - Train Loss: 0.4732, Val Loss: 0.4723, Val Accuracy: 0.8334
Epoch 6/10, cross_entropy - Train Loss: 0.4414, Val Loss: 0.4385, Val Accuracy: 0.8433
Epoch 7/10, cross_entropy - Train Loss: 0.4192, Val Loss: 0.4195, Val Accuracy: 0.8504
Epoch 8/10, cross_entropy - Train Loss: 0.4016, Val Loss: 0.4077, Val Accuracy: 0.8548
Epoch 9/10, cross_entropy - Train Loss: 0.3878, Val Loss: 0.3958, Val Accuracy: 0.8577
Epoch 10/10, cross_entropy - Train Loss: 0.3748, Val Loss: 0.3876, Val Accuracy: 0.8600


...

Epoch 1/5, cross_entropy - Train Loss: 2.2261, Val Loss: 2.1279, Val Accuracy: 0.5465
Epoch 2/5, cross_entropy - Train Loss: 2.0060, Val Loss: 1.8757, Val Accuracy: 0.6278
Epoch 3/5, cross_entropy - Train Loss: 1.7269, Val Loss: 1.5916, Val Accuracy: 0.6548
Epoch 4/5, cross_entropy - Train Loss: 1.4605, Val Loss: 1.3531, Val Accuracy: 0.6806
Epoch 5/5, cross_entropy - Train Loss: 1.2513, Val Loss: 1.1757, Val Accuracy: 0.7066


epoch,▁▃▅▆█
train_loss,█▆▄▃▁
val_accuracy,▁▅▆▇█
val_loss,█▆▄▂▁
epoch,5
loss_type,cross_entropy
train_loss,1.25129
val_accuracy,0.70658
val_loss,1.17565


wandb: Agent Starting Run: gtx958xt with config:
wandb: 	activation_function: tanh
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	hidden_layer_size: 64
wandb: 	learningg_rate_: 0.001
wandb: 	loss_type: cross_entropy
wandb: 	num_hidden_layers: 3
wandb: 	optimizer: adam
wandb: 	weight_initialization: xavier


Epoch 1/5, cross_entropy - Train Loss: 0.7273, Val Loss: 0.4399, Val Accuracy: 0.8433
Epoch 2/5, cross_entropy - Train Loss: 0.4137, Val Loss: 0.4007, Val Accuracy: 0.8562
Epoch 3/5, cross_entropy - Train Loss: 0.3730, Val Loss: 0.3776, Val Accuracy: 0.8626
Epoch 4/5, cross_entropy - Train Loss: 0.3488, Val Loss: 0.3550, Val Accuracy: 0.8724
Epoch 5/5, cross_entropy - Train Loss: 0.3321, Val Loss: 0.3563, Val Accuracy: 0.8678


epoch,▁▃▅▆█
train_loss,█▂▂▁▁
val_accuracy,▁▄▆█▇
val_loss,█▅▃▁▁
epoch,5
loss_type,cross_entropy
train_loss,0.33211
val_accuracy,0.86783
val_loss,0.35633


In [ ]:
def _main_():
    run = wandb.init(project='MA23M021_A1_Q8_MSE', reinit=True)  # Specify the project name
    config = wandb.config

    # Train the model
    weights, biases = training_model(
        X_train=X_train,
        Y_train=y_train,
        X_valid=X_valid,
        Y_valid=y_valid,
        num_hidden_layers=config.num_hidden_layers,
        num_neurons=config.hidden_layer_size,
        num_epochs=config.epochs,
        batch_size=config.batch_size,
        learningg_rate_=config.learningg_rate_,
        optimizer=config.optimizer,
        init_method=config.weight_initialization,
        activation_func=config.activation_function,
        loss_type=config.loss_type
    )

    wandb.finish()




In [ ]:
# MSE Sweep
sweep_config_mse = sweep_config.copy()
sweep_config_mse['parameters']['loss_type'] = {"value": "mse"}
print("Creating MSE Sweep...")
sweep_id_mse = wandb.sweep(sweep=sweep_config_mse, project='MA23M021_A1_Q8_MSE')
print("MSE Sweep ID:", sweep_id_mse)

Creating MSE Sweep...
Create sweep with ID: 205o72nt
Sweep URL: https://wandb.ai/ma23m021-iit-madras/MA23M021_A1_Q8_MSE/sweeps/205o72nt
MSE Sweep ID: 205o72nt


In [ ]:
wandb.agent(sweep_id_mse, function=_main_, count=50)

wandb: Agent Starting Run: qthzunpr with config:
wandb: 	activation_function: tanh
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	hidden_layer_size: 64
wandb: 	learningg_rate_: 0.001
wandb: 	loss_type: mse
wandb: 	num_hidden_layers: 5
wandb: 	optimizer: sgd
wandb: 	weight_initialization: xavier


Epoch 1/10, mse - Train Loss: 0.9089, Val Loss: 0.9000, Val Accuracy: 0.1002
Epoch 2/10, mse - Train Loss: 0.8972, Val Loss: 0.8947, Val Accuracy: 0.2579
Epoch 3/10, mse - Train Loss: 0.8919, Val Loss: 0.8888, Val Accuracy: 0.3013
Epoch 4/10, mse - Train Loss: 0.8842, Val Loss: 0.8793, Val Accuracy: 0.2509
Epoch 5/10, mse - Train Loss: 0.8727, Val Loss: 0.8663, Val Accuracy: 0.2216
Epoch 6/10, mse - Train Loss: 0.8588, Val Loss: 0.8521, Val Accuracy: 0.2563
Epoch 7/10, mse - Train Loss: 0.8447, Val Loss: 0.8384, Val Accuracy: 0.2285
Epoch 8/10, mse - Train Loss: 0.8317, Val Loss: 0.8264, Val Accuracy: 0.2550
Epoch 9/10, mse - Train Loss: 0.8206, Val Loss: 0.8164, Val Accuracy: 0.3146
Epoch 10/10, mse - Train Loss: 0.8112, Val Loss: 0.8076, Val Accuracy: 0.2859


epoch,▁▂▃▃▄▅▆▆▇█
train_loss,█▇▇▆▅▄▃▂▂▁
val_accuracy,▁▆█▆▅▆▅▆█▇
val_loss,██▇▆▅▄▃▂▂▁
epoch,10
loss_type,mse
train_loss,0.81122
val_accuracy,0.28592
val_loss,0.80758


wandb: Agent Starting Run: d08q443h with config:
wandb: 	activation_function: sigmoid
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	hidden_layer_size: 32
wandb: 	learningg_rate_: 0.001
wandb: 	loss_type: mse
wandb: 	num_hidden_layers: 4
wandb: 	optimizer: sgd
wandb: 	weight_initialization: xavier


Epoch 1/5, mse - Train Loss: 0.9134, Val Loss: 0.9055, Val Accuracy: 0.1002
Epoch 2/5, mse - Train Loss: 0.9028, Val Loss: 0.9013, Val Accuracy: 0.1002
Epoch 3/5, mse - Train Loss: 0.9005, Val Loss: 0.9001, Val Accuracy: 0.1654
Epoch 4/5, mse - Train Loss: 0.8998, Val Loss: 0.8998, Val Accuracy: 0.1676
Epoch 5/5, mse - Train Loss: 0.8997, Val Loss: 0.8996, Val Accuracy: 0.1742


...

Epoch 1/5, mse - Train Loss: 0.4511, Val Loss: 0.2703, Val Accuracy: 0.7997
Epoch 2/5, mse - Train Loss: 0.2394, Val Loss: 0.2186, Val Accuracy: 0.8482
Epoch 3/5, mse - Train Loss: 0.2050, Val Loss: 0.2059, Val Accuracy: 0.8545
Epoch 4/5, mse - Train Loss: 0.1884, Val Loss: 0.2014, Val Accuracy: 0.8607
Epoch 5/5, mse - Train Loss: 0.1792, Val Loss: 0.1839, Val Accuracy: 0.8723


epoch,▁▃▅▆█
train_loss,█▃▂▁▁
val_accuracy,▁▆▆▇█
val_loss,█▄▃▂▁
epoch,5
loss_type,mse
train_loss,0.17924
val_accuracy,0.87233
val_loss,0.18387


wandb: Agent Starting Run: cfnvlf8m with config:
wandb: 	activation_function: tanh
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	hidden_layer_size: 64
wandb: 	learningg_rate_: 0.001
wandb: 	loss_type: mse
wandb: 	num_hidden_layers: 3
wandb: 	optimizer: adam
wandb: 	weight_initialization: xavier


Epoch 1/5, mse - Train Loss: 0.3369, Val Loss: 0.2424, Val Accuracy: 0.8253
Epoch 2/5, mse - Train Loss: 0.2108, Val Loss: 0.1957, Val Accuracy: 0.8623
Epoch 3/5, mse - Train Loss: 0.1900, Val Loss: 0.1910, Val Accuracy: 0.8635
Epoch 4/5, mse - Train Loss: 0.1782, Val Loss: 0.1910, Val Accuracy: 0.8646
Epoch 5/5, mse - Train Loss: 0.1720, Val Loss: 0.1937, Val Accuracy: 0.8655


epoch,▁▃▅▆█
train_loss,█▃▂▁▁
val_accuracy,▁▇███
val_loss,█▂▁▁▁
epoch,5
loss_type,mse
train_loss,0.17196
val_accuracy,0.8655
val_loss,0.19374
